In [28]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler # Import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from scipy.stats import mode
from sklearn.metrics import f1_score
# ... (rest of your code) ...




In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load the dataset
path = '/content/drive/My Drive/University/Final Project/dataset.csv'
initial_data = pd.read_csv(path)

In [5]:
initial_data.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [6]:
# Drop unnecessary columns
mydata = initial_data.drop(['Person ID', 'Occupation', 'Sleep Disorder'], axis=1, errors='ignore')

In [7]:
# Display the cleaned data
print("Cleaned Data:")
mydata.head()

Cleaned Data:


,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps
0,Male,27,6.1,6,42,6,Overweight,126/83,77,4200
1,Male,28,6.2,6,60,8,Normal,125/80,75,10000
2,Male,28,6.2,6,60,8,Normal,125/80,75,10000
3,Male,28,5.9,4,30,8,Obese,140/90,85,3000
4,Male,28,5.9,4,30,8,Obese,140/90,85,3000


In [8]:
# Encode categorical variables
X = mydata.drop('Stress Level', axis=1)
y = mydata['Stress Level']
X = pd.get_dummies(X, drop_first=True)

In [9]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
rf_pred = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)

In [11]:
# SVM
svm_classifier = SVC(probability=True, kernel='rbf', random_state=42)
svm_classifier.fit(X_train, y_train)
svm_pred = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_pred)

In [12]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder to your target variable and transform it
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# XGBoost Classifier
xgb_classifier = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(X_train, y_train)
xgb_pred = xgb_classifier.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_pred)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:32:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [13]:
# Logistic Regression
logreg_classifier = LogisticRegression(max_iter=1000, random_state=42)
logreg_classifier.fit(X_train, y_train)
logreg_pred = logreg_classifier.predict(X_test)
logreg_accuracy = accuracy_score(y_test, logreg_pred)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
# Display all model accuracies
print("\nModel Accuracies:")
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")
print(f"SVM Accuracy: {svm_accuracy:.2f}")
print(f"XGBoost Accuracy: {xgb_accuracy:.2f}")
print(f"Logistic Regression Accuracy: {logreg_accuracy:.2f}")



Model Accuracies:
Random Forest Accuracy: 1.00
SVM Accuracy: 0.41
XGBoost Accuracy: 1.00
Logistic Regression Accuracy: 0.89


In [15]:
# Define recommendation and action plan functions
def content_based_recommendation(stress_level, physical_activity, sleep_quality, bmi, blood_pressure):
    if stress_level > 7:
        return "Meditation, Yoga, Reduce screen time, Regular Sleep"
    elif physical_activity < 5:
        return "Increase Physical Activity, Morning Walk, Breathing Exercises"
    elif sleep_quality < 5:
        return "Improve Sleep Hygiene, Avoid Caffeine, Create Sleep Schedule"
    elif bmi > 30:
        return "Consult a nutritionist, Begin a weight loss program"
    elif blood_pressure > 140:
        return "Consult with a healthcare provider for blood pressure management"
    else:
        return "Maintain Current Lifestyle, Stay Positive"

In [16]:
def suicide_action_plan(stress_level, sleep_quality, physical_activity, daily_steps, bmi, blood_pressure):
    # Critical Risk Conditions: Stress level is very high or multiple lifestyle factors are poor
    if stress_level > 8 or (
        (sleep_quality < 2 and physical_activity < 2 and daily_steps < 1000) or
        (bmi > 30 and blood_pressure > 140) or
        (sleep_quality < 3 and physical_activity < 3 and daily_steps < 2000)
    ):
        return "Critical Risk: Immediate professional intervention is required. Contact mental health professionals and emergency helplines now."

    # High Risk Conditions: Stress level is high or major lifestyle issues are present
    elif stress_level > 7 or ((sleep_quality < 3 and physical_activity < 3) or (bmi > 28 and blood_pressure > 130)):
        return "High Risk: Seek guidance from a licensed psychologist or psychiatrist. Immediate attention is required."

    # Moderate Risk Conditions: Stress is moderate and some lifestyle factors are suboptimal
    elif stress_level > 5 or (
        (sleep_quality < 4 and physical_activity < 5) or
        (bmi > 25 and blood_pressure > 120)
    ):
        return "Moderate Risk: Practice relaxation techniques, mindfulness, and yoga. Consult a counselor to manage stress."

    # Low Risk Conditions: Stress is low, but lifestyle factors should be maintained
    else:
        return "Low Risk: Continue maintaining a balanced lifestyle with adequate sleep, regular physical activity, and social support."


In [ ]:
print("\nXGBoost Results:")
print(f"Accuracy: {xgb_accuracy:.2f}")
print(classification_report(y_test, xgb_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, xgb_pred))


XGBoost Results:
Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        12
           5       1.00      1.00      1.00        17

    accuracy                           1.00        75
   macro avg       1.00      1.00      1.00        75
weighted avg       1.00      1.00      1.00        75


Confusion Matrix:
 [[12  0  0  0  0  0]
 [ 0 10  0  0  0  0]
 [ 0  0 14  0  0  0]
 [ 0  0  0 10  0  0]
 [ 0  0  0  0 12  0]
 [ 0  0  0  0  0 17]]


In [29]:
# Create a StandardScaler instance and fit it to your training data
scaler = StandardScaler()  # Create a StandardScaler object
scaler.fit(X_train)  # Fit the scaler to your training data

StandardScaler()

In [24]:
# Function to test a single person's data
def test_single_person(gender, age, sleep_duration, physical_activity, bmi, blood_pressure, heart_rate, daily_steps):

    # Single input data (Modified to match original dataset structure)
    person_data = pd.DataFrame({
        'Gender': [gender],
        'Age': [age],
        'Sleep Duration': [sleep_duration],
        'Physical Activity Level': [physical_activity],
        'BMI': [bmi],
        'Blood Pressure': [blood_pressure],
        'Heart Rate': [heart_rate],
        'Daily Steps': [daily_steps]
    }, index=[0])  # Adding an index to avoid potential issues

    # Preprocess person_data to match the format used during training:
    # 1. One-hot encode categorical features
    person_data = pd.get_dummies(person_data, columns=['Gender'], drop_first=True)

    # 2. Ensure all columns from training data are present
    #    (Add missing columns with 0 values)
    for col in X_train.columns:  # X_train has the expected columns
        if col not in person_data.columns:
            person_data[col] = 0

    # 3. Reorder columns to match the order in training data
    person_data = person_data[X_train.columns]

    # Now you can safely scale the data:
    person_scaled = scaler.transform(person_data)

    # Predict stress level using the models
    rf_pred = rf_classifier.predict(person_data)[0]
    svm_pred = svm_classifier.predict(person_data)[0]
    xgb_pred = xgb_classifier.predict(person_data)[0]
    logreg_pred = logreg_classifier.predict(person_data)[0]
    # Combine predictions using voting (majority voting for categorical output)
    predictions = [rf_pred, svm_pred, xgb_pred, logreg_pred]
    final_pred = max(set(predictions), key=predictions.count)  # Majority voting

    # Get stress relief recommendations
    recommendation = content_based_recommendation(
        stress_level=final_pred,  # Using the combined prediction
        physical_activity=physical_activity,
        sleep_quality=sleep_duration,
        bmi=bmi,
        blood_pressure=blood_pressure
    )

    # Suicide action plan
    action_plan = suicide_action_plan(
        stress_level=final_pred,  # Using the combined prediction
        sleep_quality=sleep_duration,
        physical_activity=physical_activity,
        daily_steps=daily_steps,
        bmi=bmi,
        blood_pressure=blood_pressure
    )

    # Output the results
    print(f"\nTest for Single Person's Data:")
    print(f"Gender: {gender}")
    print(f"Age: {age}")
    print(f"Sleep Duration: {sleep_duration} hours")
    print(f"Physical Activity Level: {physical_activity}")
    print(f"BMI: {bmi}")
    print(f"Blood Pressure: {blood_pressure} mmHg")
    print(f"Heart Rate: {heart_rate} bpm")
    print(f"Daily Steps: {daily_steps}")
    print(f"Predicted Stress Level (Random Forest): {rf_pred}")
    print(f"Predicted Stress Level (SVM): {svm_pred}")
    print(f"Predicted Stress Level (XGBoost): {xgb_pred}")
    print(f"Predicted Stress Level (Logistic Regression): {logreg_pred}")
    test_and_predict_stress_level_with_meta_model(gender, age, sleep_duration, physical_activity, bmi, blood_pressure, heart_rate, daily_steps)

In [18]:
def get_base_model_predictions(X_train):
    rf_probs = rf_classifier.predict_proba(X_train)
    svm_probs = svm_classifier.predict_proba(X_train)
    xgb_probs = xgb_classifier.predict_proba(X_train)
    # Combine probabilities for the meta-model
    meta_features = np.hstack((rf_probs, svm_probs, xgb_probs))
    return meta_features

In [19]:
def test_and_predict_stress_level_with_meta_model(
    gender, age, sleep_duration, physical_activity, bmi, blood_pressure, heart_rate, daily_steps
):

    # Single input data (Modified to match original dataset structure)
    person_data = pd.DataFrame({
        'Gender': [gender],
        'Age': [age],
        'Sleep Duration': [sleep_duration],
        'Physical Activity Level': [physical_activity],
        'BMI': [bmi],
        'Blood Pressure': [blood_pressure],
        'Heart Rate': [heart_rate],
        'Daily Steps': [daily_steps]
    })

    # Preprocess the input to match training data
    person_data = pd.get_dummies(person_data, columns=['Gender'], drop_first=True)

    # Add missing columns with 0 values
    for col in X_train.columns:
        if col not in person_data.columns:
            person_data[col] = 0

    # Reorder columns to match the training data
    person_data = person_data[X_train.columns]

    # Scale the input data
    person_scaled = scaler.transform(person_data)

    # Get predictions from base models
    rf_probs = rf_classifier.predict_proba(person_scaled)
    svm_probs = svm_classifier.predict_proba(person_scaled)
    xgb_probs = xgb_classifier.predict_proba(person_scaled)

    # Combine probabilities for the meta-model
    meta_features = np.hstack((rf_probs, svm_probs, xgb_probs))

    meta_model = LogisticRegression(max_iter=1000, random_state=42)
    meta_model.fit(get_base_model_predictions(X_train), y_train)

    # Meta-model prediction
    meta_pred = meta_model.predict(meta_features)


    # Recommendations based on stress level
    recommendation = content_based_recommendation(
        stress_level=meta_pred[0],  # Use combined prediction
        physical_activity=physical_activity,
        sleep_quality=sleep_duration,
        bmi=bmi,
        blood_pressure=blood_pressure
    )

    # Suicide action plan based on stress level
    action_plan = suicide_action_plan(
        stress_level=meta_pred[0],  # Use combined prediction
        sleep_quality=sleep_duration,
        physical_activity=physical_activity,
        daily_steps=daily_steps,
        bmi=bmi,
        blood_pressure=blood_pressure
    )

    # Output the results
    print(f"\nMeta Model:")
    print(f"Predicted Stress Level (Meta-Model): {meta_pred[0]}")
    print(f"Recommendations: {recommendation}")
    print(f"Suicide Action Plan: {action_plan}")


In [21]:
# Calculate F1 scores
rf_f1_score = f1_score(y_test, rf_pred, average='weighted')
svm_f1_score = f1_score(y_test, svm_pred, average='weighted')
xgb_f1_score = f1_score(y_test, xgb_pred, average='weighted')
logreg_f1_score = f1_score(y_test, logreg_pred, average='weighted')

# Get meta_pred for F1 score calculation
# Assuming you want to use the same test data (X_test) for meta-model prediction
meta_features_test = get_base_model_predictions(X_test)
meta_model = LogisticRegression(max_iter=1000, random_state=42)
meta_model.fit(get_base_model_predictions(X_train), y_train)  # Fit meta-model
meta_pred = meta_model.predict(meta_features_test)  # Predict using test data

meta_model_f1_score = f1_score(y_test, meta_pred, average='weighted')

# Display F1 scores
print("\nF1 Scores:")
print(f"Random Forest F1 Score: {rf_f1_score:.2f}")
print(f"SVM F1 Score: {svm_f1_score:.2f}")
print(f"XGBoost F1 Score: {xgb_f1_score:.2f}")
print(f"Logistic Regression F1 Score: {logreg_f1_score:.2f}")
print(f"Meta Model F1 Score: {meta_model_f1_score:.2f}")


F1 Scores:
Random Forest F1 Score: 0.00
SVM F1 Score: 0.11
XGBoost F1 Score: 1.00
Logistic Regression F1 Score: 0.90
Meta Model F1 Score: 1.00


In [22]:
# Display Confusion Matrices
print("\nConfusion Matrices:")
print("Random Forest Confusion Matrix:")
print(confusion_matrix(y_test, rf_pred))

print("\nSVM Confusion Matrix:")
print(confusion_matrix(y_test, svm_pred))

print("\nXGBoost Confusion Matrix:")
print(confusion_matrix(y_test, xgb_pred))

print("\nLogistic Regression Confusion Matrix:")
print(confusion_matrix(y_test, logreg_pred))

print("\nMeta Model Confusion Matrix:")
print(confusion_matrix(y_test, meta_pred))


Confusion Matrices:
Random Forest Confusion Matrix:
[[ 0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0]
 [ 0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  0  0 12  0]
 [ 0  0  0  0  0  0  0  0 17]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]

SVM Confusion Matrix:
[[ 0  0  0  6  6  0  0]
 [ 0  0  0  1  9  0  0]
 [ 0  0  0  2  1 11  0]
 [ 0  0  0  1  1  8  0]
 [ 0  0  0  1 11  0  0]
 [ 0  0  0 11  0  1  5]
 [ 0  0  0  0  0  0  0]]

XGBoost Confusion Matrix:
[[12  0  0  0  0  0]
 [ 0 10  0  0  0  0]
 [ 0  0 14  0  0  0]
 [ 0  0  0 10  0  0]
 [ 0  0  0  0 12  0]
 [ 0  0  0  0  0 17]]

Logistic Regression Confusion Matrix:
[[12  0  0  0  0  0]
 [ 0  9  1  0  0  0]
 [ 0  0 11  3  0  0]
 [ 0  0  0  9  1  0]
 [ 0  0  0  1 11  0]
 [ 0  0  0  2  0 15]]

Meta Model Confusion Matrix:
[[12  0  0  0  0  0]
 [ 0 10  0  0  0  0]
 [ 0  0 14  0  0  0]
 [ 0  0  0 10  0  0]
 [ 0  0  0  0 12  0]
 [ 0  0  0  0  0 17]]


In [30]:
test_single_person(
    gender='Male',             # Gender: 'Male' or 'Female'
    age=24,                    # Age in years
    sleep_duration=3.2,          # Sleep Duration in hours
    physical_activity=3,       # Physical Activity level (1-10)
    bmi=17.2,                    # Body Mass Index
    blood_pressure=132,        # Blood Pressure in mmHg (e.g., 120/80)
    heart_rate=96,             # Heart Rate in beats per minute
    daily_steps=10000           # Daily Steps count
)


Test for Single Person's Data:
Gender: Male
Age: 24
Sleep Duration: 3.2 hours
Physical Activity Level: 3
BMI: 17.2
Blood Pressure: 132 mmHg
Heart Rate: 96 bpm
Daily Steps: 10000
Predicted Stress Level (Random Forest): 8
Predicted Stress Level (SVM): 8
Predicted Stress Level (XGBoost): 5
Predicted Stress Level (Logistic Regression): 5

Meta Model:
Predicted Stress Level (Meta-Model): 5
Recommendations: Increase Physical Activity, Morning Walk, Breathing Exercises
Suicide Action Plan: Moderate Risk: Practice relaxation techniques, mindfulness, and yoga. Consult a counselor to manage stress.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
